In [ ]:
from uchronia.wrap.uchronia_wrap_generated import *

In [ ]:
import os

## -----------------------------------------------------------------------------
sampleDataDir = "/home/per202/data/documentation"
dataLibraryFile = os.path.join(sampleDataDir, 'time_series_library.yaml')

In [ ]:
## -----------------------------------------------------------------------------
dataLibrary = LoadEnsembleDataset_py(dataLibraryFile, '')
dataIds = GetEnsembleDatasetDataIdentifiers_py(dataLibrary)
dataIds


In [ ]:
def getDataSet(dataLibrary, dataIdentifier):
    desc = GetDataDimensionsDescription_py(dataLibrary, dataIdentifier)
    dimensions = len(desc)
    if dimensions == 1:
        return GetDatasetSingleTimeSeries_py(dataLibrary, dataIdentifier)
    if dimensions == 2:
        return GetDatasetEnsembleTimeSeries_py(dataLibrary, dataIdentifier)
    if dimensions == 3:
        return GetDatasetEnsembleForecastTimeSeries_py(dataLibrary, dataIdentifier)
    else:
        raise KeyError("Number of dimensions for a data set is not supported: " + str(dimensions))


In [ ]:
singleObs = getDataSet(dataLibrary, "var1_obs")

In [ ]:
singleObs

In [ ]:
stationIds = GetEnsembleDatasetDataSubIdentifiers_py(dataLibrary, "var1_obs_collection")
print(stationIds)

In [ ]:
ToStructSingleTimeSeriesData_py(singleObs)

In [ ]:
singleObs

In [ ]:
ToStructSingleTimeSeriesData_py(singleObs)

In [ ]:
multiObs = getDataSet(dataLibrary, "var1_obs_collection")
multiObs


In [ ]:
ToStructEnsembleTimeSeriesData_py(multiObs)